In [ ]:
import pandas as pd
import textblob
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
import string

In [ ]:
df=pd.read_csv("temp_data3.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df["Sentiment"].value_counts()

In [ ]:
veri=df["entire_text"].apply(lambda x: " ".join(x.lower() for x in x.split()))# büyük harfler küçüğe dönüştürüldü

In [ ]:
veri.head()

In [ ]:
veri2=veri.str.replace("[^\w\s]","")# harf dışı simgeler ve noktalama silindi
#Noktalama işaretleri başka yöntem->noktalama=string.punctuation

In [ ]:
veri2.head()

In [ ]:
nltk.download("stopwords")
duraklama_kelimeleri = stopwords.words()
veri3=pd.DataFrame(veri2,columns=["entire_text"])
veri4=veri3["entire_text"].apply(lambda x: " ".join(x for x in x.split() if x not in duraklama_kelimeleri))

In [ ]:
for i in veri4["entire_text"]:
    print(i,"\n")

In [ ]:
veri4 = pd.DataFrame(veri4, columns = ["entire_text"])# veri4 Series olduğundan
pd.Series(" ".join(veri4["entire_text"]).split()).value_counts()
#En az geçen kelimeleri sıralayıp en sondaki kelime çıkarıldı
az = pd.Series(" ".join(veri4["entire_text"]).split()).value_counts()[-1:]
veri5 = veri4["entire_text"].apply(lambda x: " ".join(i for i in x.split() if i not in az)) 

#En baş yani en çok geçen 3 kelime
cok = pd.Series(" ".join(veri4["entire_text"]).split()).value_counts()[0:3] 
veri6 = veri4["entire_text"].apply(lambda x: " ".join(i for i in x.split() if i not in cok))

In [ ]:
veri6.head()

In [ ]:
sentiment=df["Sentiment"]
veri7=pd.concat([veri6,sentiment],axis=1)

In [ ]:
veri7.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(veri7["entire_text"].values.astype('U'),
                                              veri7["Sentiment"].values.astype('U'),test_size=0.1,random_state=42)
#astype('U')-> verileri dizeye dönüştürür
print(X_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()
X_train_counts=count_vect.fit_transform(X_train)#yorumlarda geçen kelimelerin sayısını tutan vektör oluşturulur,sayma vektörü
print(X_tarin_counts.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer()
X_train_tfidf=tfidf_transformer.fit_transform(X_train_counts)#yorumu temsil edecek,diğer yorumlarda sıklıkla geçmeyen kelimeler 
print(X_train_tfidf.shape)# yorumsayısı, kelimesayısı
#tf kelimenin metinde kaç kere geçtiğini, idf bir kelimenin kaç dokümanda geçtiğini verir,tfidf bir kelimenin doküman için önemi

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB().fit(X_train_tfidf,y_train)#y_train->etiketler
X_test_counts=count_vect.transform(X_test)
X_test_tfidf=tfidf_transformer.transform(X_test_counts)#test için ayrılan veri yorumlarının tfidf çevirimi
print(X_test_tfidf.shape)

In [ ]:
y_pred=clf.predict(X_test_tfidf)
for review, sentiment in zip(X_test[:5],y_pred[:]):
    print('%r ---> %s'%(review,sentiment))

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))